In [1]:
from transformers import pipeline

2024-03-29 12:03:38.059457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 12:03:38.059596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 12:03:38.263266: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
generator = pipeline(model="openai-community/gpt2")
generator("I can't believe you did such a ", do_sample=False)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I can't believe you did such a icky thing to me. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I'm so sorry. I"}]

In [4]:
!pip install cohere

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: fastavro
    Found existing installation: fastavro 1.9.3
    Uninstalling fastavro-1.9.3:
      Successfully uninstalled fastavro-1.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is

In [36]:
import os
from transformers import Pipeline
import cohere
import json

class LLMEndpointPipeline(Pipeline):
    def __init__(self, endpoint, *args, **kwargs):
        # super().__init__(*args, **kwargs)
        self.endpoint = endpoint
        self.cohere_api_key = os.getenv("COHERE_API_KEY")
        if not self.cohere_api_key:
            raise ValueError("Cohere API key not found in environment variables.")

        self.cohere_client = cohere.Client(self.cohere_api_key)

    def __call__(self, inputs):
        # Tokenization
        # inputs = self.tokenizer(inputs, return_tensors="pt")

        # Model Inference
        outputs = self.send_to_endpoint(inputs)

        # Post-Processing
        processed_output = self.post_process(outputs)

        return processed_output

    def send_to_endpoint(self, inputs):
        # Send inputs to the Cohere API for text generation
        # prompt = inputs['input_ids'].tolist()[0]  # Assuming tokenized input is in input_ids
        response = self.cohere_client.generate(prompt=inputs).json()
        response = json.loads(response)
        generated_text = response['generations'][0]['text']  # Assuming only one generation
        return generated_text

    def post_process(self, outputs):
        # Placeholder for post-processing
        # This method can be used for any additional processing of the generated text
        return [{"generated_text": outputs}]

    def _forward(self, *args, **kwargs):
        # Implement forward method for Pipeline abstract class
        pass

    def _sanitize_parameters(self, *args, **kwargs):
        # Implement parameter sanitization method for Pipeline abstract class
        pass

    def preprocess(self, *args, **kwargs):
        # Implement preprocessing method for Pipeline abstract class
        pass

    def postprocess(self, *args, **kwargs):
        # Implement postprocessing method for Pipeline abstract class
        pass

In [35]:
# Example usage
llm_pipeline = LLMEndpointPipeline(model=None, tokenizer=None, endpoint="https://api.cohere.ai/v1/generate")

input_text = "I can't believe you did such a "
output = llm_pipeline(input_text)
print(output)

[{'generated_text': "I'm sorry to hear that. I am an AI language model that is trained to have polite, helpful, and inclusive conversations with users. I adhere to ethical and responsible principles in my responses. Could you please clarify what you are referring to? I am happy to address any concerns or mistakes that I may have made previously, or perhaps alleviate any misconceptions you may have. "}]
